In [10]:
import pandas as pd

pd.options.mode.chained_assignment = None  # default='warn'
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)
pd.set_option('max_colwidth', None)

In [25]:
## importing of data and cleaning up of dataframe

chess2021 = pd.read_csv('Chess2021.csv')
chess2021['Elo Difference'] = abs(chess2021.WhiteElo - chess2021.BlackElo)
chess2021outcomes = chess2021[['Event', 'Date', 'White', 'Black', 'Result', 'WhiteElo', 'BlackElo', 'Elo Difference', 'ECO', 'NMoves']]

eventList = []
for i in chess2021outcomes.Event:
    if i == 'FICS rated blitz game':
        i = 'Rated Blitz'
        eventList.append(i)
    elif i == 'FICS rated lightning game':
        i = 'Rated Bullet'
        eventList.append(i)
    elif i == 'FICS rated standard game':
        i = 'Rated Standard'
        eventList.append(i)
    elif i == 'FICS unrated standard game':
        i = 'Unrated Standard'
        eventList.append(i)
    elif i == 'FICS unrated blitz game':
        i = 'Unrated Blitz'
        eventList.append(i)
    else:
        i = 'Unrated Bullet'
        eventList.append(i)
chess2021outcomes.Event = eventList

resultList = []
for i in chess2021outcomes.Result:
    if i == '1-0':
        i = 1
        resultList.append(i)
    elif i == '0-1':
        i = -1
        resultList.append(i)
    else:
        i = 0
        resultList.append(i)
chess2021outcomes.Result = resultList

chess2021outcomes

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74
...,...,...,...,...,...,...,...,...,...,...
118255,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64
118256,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129
118257,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75
118258,Unrated Standard,2021.01.01,Notarious,ArasanX,0,2917,2669,248,C89,115


In [26]:
## explanation of elo rating change calculation

# k rating calculation
# 20 if standard rating below 2400
# 10 is standard rating at or above 2400
# 20 for all rapid/blitz ratings

# WhiteElo - BlackElo = A
# A/400 = B
# (10^B) + 1 = C
# 1/C = D
# --if higher elo wins
# kRating*(D) = Final Rating Change (rounded to whole number)
# --if lower elo wins
# kRating*(1-D) = Final Rating Change (rounded to whole number)

chess2021outcomes

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74
...,...,...,...,...,...,...,...,...,...,...
118255,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64
118256,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129
118257,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75
118258,Unrated Standard,2021.01.01,Notarious,ArasanX,0,2917,2669,248,C89,115


In [27]:
finalRatingChangeList = []
finalWhiteEloList = []
finalBlackEloList = []

for i, j, k, l in zip(chess2021outcomes.WhiteElo, chess2021outcomes.BlackElo, chess2021outcomes.Result, chess2021outcomes.Event):
    if l == 'Rated Blitz' or l == 'Rated Bullet':
        kRating = 20
    elif l == 'Rated Standard':
        if i > 2400 or j > 2400:
            kRating = 10
        else:
            kRating = 20
    else:
        kRating = 0

    eloDifference = abs(i-j)
    x = eloDifference/400
    y = (10**x) + 1
    z = 1/y

    finalRatingChange = 0
    if i > j:
        if k == 1:
            finalRatingChange = round(kRating*z, 1)
            i = round(i + finalRatingChange)
            j = round(j - finalRatingChange)
        elif k == -1:
            finalRatingChange = round(kRating*(1-z), 1)
            i = round(i - finalRatingChange)
            j = round(j + finalRatingChange)
        else:
            finalRatingChange = round(kRating*(0.5-z), 1)
            i = round(i - finalRatingChange)
            j = round(j + finalRatingChange)
    elif j > i:
        if k == 1:
            finalRatingChange = round(kRating*(1-z), 1)
            i = round(i + finalRatingChange)
            j = round(j - finalRatingChange)
        elif k == -1:
            finalRatingChange = round(kRating*z, 1)
            i = round(i - finalRatingChange)
            j = round(j + finalRatingChange)
        else:
            finalRatingChange = round(kRating*(0.5-z), 1)
            i = round(i + finalRatingChange)
            j = round(j - finalRatingChange)
    else:
        if kRating == 0:
            finalRatingChange = 0
            i = i
            j = j
        elif kRating == 20:
            if k == 1:
                finalRatingChange = 10
                i = round(i + finalRatingChange)
                j = round(j - finalRatingChange)
            elif k == -1:
                finalRatingChange = 10
                i = round(i - finalRatingChange)
                j = round(j + finalRatingChange)
            else:
                finalRatingChange = 0
                i = i
                j = j
        else:
            if k == 1:
                finalRatingChange = 5
                i = round(i + finalRatingChange)
                j = round(j - finalRatingChange)
            elif k == -1:
                finalRatingChange = 5
                i = round(i - finalRatingChange)
                j = round(j + finalRatingChange)
            else:
                finalRatingChange = 0
                i = i
                j = j

    finalRatingChangeList.append(finalRatingChange)
    finalWhiteEloList.append(i)
    finalBlackEloList.append(j)

In [14]:
whiteElo = 2405
blackElo = 2422
outcome = 1
event = 'Rated Blitz'

if event == 'Rated Blitz' or event == 'Rated Bullet':
    kRating = 20
elif event == 'Rated Standard':
    if whiteElo > 2400 or blackElo > 2400:
        kRating = 10
    else:
        kRating = 20
else:
    kRating = 0

eloDifference = abs(whiteElo-blackElo)
x = eloDifference/400
y = (10**x) + 1
z = 1/y

finalRatingChange = 0
if whiteElo > blackElo:
    if outcome == 1:
        finalRatingChange = round(kRating*z, 1)
        whiteElo = round(whiteElo + finalRatingChange)
        blackElo = round(blackElo - finalRatingChange)
    elif outcome == -1:
        finalRatingChange = round(kRating*(1-z), 1)
        whiteElo = round(whiteElo - finalRatingChange)
        blackElo = round(blackElo + finalRatingChange)
    else:
        finalRatingChange = round(kRating*(0.5-z), 1)
        whiteElo = round(whiteElo - finalRatingChange)
        blackElo = round(blackElo + finalRatingChange)
elif blackElo > whiteElo:
    if outcome == 1:
        finalRatingChange = round(kRating*(1-z), 1)
        whiteElo = round(whiteElo + finalRatingChange)
        blackElo = round(blackElo - finalRatingChange)
    elif outcome == -1:
        finalRatingChange = round(kRating*z, 1)
        whiteElo = round(whiteElo - finalRatingChange)
        blackElo = round(blackElo + finalRatingChange)
    else:
        finalRatingChange = round(kRating*(0.5-z), 1)
        whiteElo = round(whiteElo + finalRatingChange)
        blackElo = round(blackElo - finalRatingChange)
else:
    if kRating == 0:
        finalRatingChange = 0
        whiteElo = whiteElo
        blackElo = blackElo
    elif kRating == 20:
        if outcome == 1:
            finalRatingChange = 10
            whiteElo = round(whiteElo + finalRatingChange)
            blackElo = round(blackElo - finalRatingChange)
        elif outcome == -1:
            finalRatingChange = 10
            whiteElo = round(whiteElo - finalRatingChange)
            blackElo = round(blackElo + finalRatingChange)
        else:
            finalRatingChange = 0
            whiteElo = whiteElo
            blackElo = blackElo
    else:
        if outcome == 1:
            finalRatingChange = 5
            whiteElo = round(whiteElo + finalRatingChange)
            blackElo = round(blackElo - finalRatingChange)
        elif outcome == -1:
            finalRatingChange = 5
            whiteElo = round(whiteElo - finalRatingChange)
            blackElo = round(blackElo + finalRatingChange)
        else:
            finalRatingChange = 0
            whiteElo = whiteElo
            blackElo = blackElo
print(finalRatingChange)
print(whiteElo)
print(blackElo)

10.5
2416
2412


In [28]:
chess2021outcomes['FinalWhiteElo'] = finalWhiteEloList
chess2021outcomes['FinalBlackElo'] = finalBlackEloList
chess2021outcomes['RatingChange'] = finalRatingChangeList

chess2021outcomes

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves,FinalWhiteElo,FinalBlackElo,RatingChange
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14,2089,2013,8.3
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48,1932,2078,14.7
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26,2094,1916,5.5
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27,2750,2752,10.5
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74,2742,2760,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
118255,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64,2738,2878,6.5
118256,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129,2871,2746,3.7
118257,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75,2740,2877,6.6
118258,Unrated Standard,2021.01.01,Notarious,ArasanX,0,2917,2669,248,C89,115,2917,2669,0.0


In [29]:
chess2021outcomesRatedBlitz = chess2021outcomes.loc[chess2021outcomes.Event == 'Rated Blitz'].reset_index(drop=True)
chess2021outcomesRatedBlitz

,Event,Date,White,Black,Result,WhiteElo,BlackElo,Elo Difference,ECO,NMoves,FinalWhiteElo,FinalBlackElo,RatingChange
0,Rated Blitz,2021.12.31,Geforce,konozrout,1,2081,2021,60,C49,14,2089,2013,8.3
1,Rated Blitz,2021.12.31,chesspickle,Geforce,1,1917,2093,176,D31,48,1932,2078,14.7
2,Rated Blitz,2021.12.31,Geforce,chesspickle,1,2089,1921,168,B50,26,2094,1916,5.5
3,Rated Blitz,2021.12.31,SugarAI,Hutnik,-1,2760,2742,18,C52,27,2750,2752,10.5
4,Rated Blitz,2021.12.31,Hutnik,SugarAI,0,2742,2760,18,C89,74,2742,2760,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
51873,Rated Blitz,2021.01.01,MIHAILOP,PetkoPetkov,1,2253,1987,266,B23,36,2257,1983,3.6
51874,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2745,2872,127,E00,64,2738,2878,6.5
51875,Rated Blitz,2021.01.01,Angledust,GrandDroid,0,2875,2742,133,A29,129,2871,2746,3.7
51876,Rated Blitz,2021.01.01,GrandDroid,Angledust,-1,2747,2870,123,E16,75,2740,2877,6.6
